<a href="https://colab.research.google.com/github/elainedias16/TCC/blob/main/Roteiro_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
from bs4 import BeautifulSoup

def remove_tags(html):
    # parse html content
    soup = BeautifulSoup(html, "html.parser")

    for data in soup(['style', 'script']):
        # Remove tags
        data.decompose()

    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)


#Stemming
import nltk
from nltk.stem import RSLPStemmer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('rslp')

# Inicializar o stemmer RSLPStemmer
stemmer = RSLPStemmer()

def stemming(text):
  words = word_tokenize(text)
  stemmed_words = [stemmer.stem(word) for word in words]
  stemmed_sentence = ' '.join(stemmed_words)
  return stemmed_sentence


In [ ]:
import unicodedata
import re
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('portuguese'))

def pre_processing(text):
  #Put space in non alphanumeric characteres
  text_without_non_alnum = ''
  for character in text:
    if character.isalnum():
      text_without_non_alnum += character
    else:
      text_without_non_alnum += ' '

  #Remove accents:
  text_without_accents = ''.join(c for c in unicodedata.normalize('NFD', text_without_non_alnum) if unicodedata.category(c) != 'Mn')

  #Remove duplicated spaces:
  text_without_duplicates = ' '.join(text_without_accents.split())

  #Lowercasting
  text_without_duplicates = text_without_duplicates.lower()

  #Remove stop words:
  words = nltk.word_tokenize(text_without_duplicates)
  text_without_stop_words = ''
  for word in words:
    if word not in stop_words:
      text_without_stop_words += word + ' ' #It's necessary to put space because tokenize takes away it

  #Stemming:
  stemmed_text = stemming(text_without_stop_words)



  # return text_without_stop_words
  return stemmed_text



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
###teste sem append
from posixpath import defpath
import requests
import pandas as pd
import os
import chardet

id_path = '1TJTEKu5hxk25q36O6xF1OruLjySlW6_n'
directory_path = '/content/drive/MyDrive/dados-aTribuna-classes-selecionadas'

import pandas as pd
from bs4 import BeautifulSoup

def remove_tags(html):
    # parse html content
    soup = BeautifulSoup(html, "html.parser")
    for data in soup(['style', 'script']):
        # Remove tags
        data.decompose()
    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)

def create_dataframe():
  df = pd.DataFrame(columns=['genre', 'content'])
  folders = os.listdir(directory_path)

  for folder in folders:
    genre_path = directory_path + '/' + folder

    if os.path.isdir(genre_path): #There is a file in the root
        os.chdir(genre_path)
        for filename in os.listdir(genre_path):
            file_path = os.path.join(genre_path, filename)

            if filename.endswith('.txt'):
              with open(file_path, 'rb') as file:
                content = file.read()
                content_without_html_tags = remove_tags(content)
                content_pre_procesed = pre_processing(content_without_html_tags)
                # df = df.append({'genre': folder, 'content': content_pre_procesed}, ignore_index=True)
                df_to_concat = pd.DataFrame({'genre': folder, 'content': content_pre_procesed}, index=[0])
                df = pd.concat([df, df_to_concat], ignore_index=True)




  return df


# print(create_dataframe())
df = create_dataframe()

df.head()

,genre,content
0,cit,religiao grande cruzada evangelistica itaciba ...
1,cit,protecao abrolhos vira caso judicial governo b...
2,cit,bebida adolescencia afeta cerebro quanto alcoo...
3,cit,soja contra envelhecimento cientistas descobri...
4,cit,decifrado dna abelhas pesquisas ajudam entende...


In [23]:
# !pip install spacy-model-pt_core_news_sm

import spacy

nlp = spacy.load('en_core_web_sm')
# nlp = spacy.load('pt_core_news_sm')

text = "The quick brown foxes are jumping over the lazy dogs."

def lemmanization(text):
  doc = nlp(text)
  lemmatized_tokens = [token.lemma_ for token in doc]
  lemmatized_text = ' '.join(lemmatized_tokens)
  return lemmatized_text

print("Original Text:", text)
print("Lemmatized Text:", lemmanization(text))


Original Text: The quick brown foxes are jumping over the lazy dogs.
Lemmatized Text: the quick brown fox be jump over the lazy dog .
